<a href="https://colab.research.google.com/github/zackbatch/BA820-Project/blob/main/Varun_EDA_BA820_project_Group_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PLAY-BY-PLAY, PLAYER-BY-PLAYER – COLLEGE BASKETBALL ANALYTICS**

**Group 12 Team Members:**
Audrey Sellers,
Jaishankar Govindaraj,
Priyanka Chaudhari,
Varun Kaza,
Zack Batchelder

#Importing Libraries

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import gensim.downloader as api
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [ ]:
#uploading the dataset
df = pd.read_csv('https://raw.githubusercontent.com/zackbatch/BA820-Project/main/CollegeBasketballPlayers2009-2021.csv')
df

<ipython-input-2-310eda7333dd>:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://raw.githubusercontent.com/zackbatch/BA820-Project/main/CollegeBasketballPlayers2009-2021.csv')


,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,Pure PG,0.00000
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,C,0.00000
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-1.126810,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-2.380750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000


In [ ]:
dftext = pd.read_csv("https://raw.githubusercontent.com/zackbatch/BA820-Project/main/Instant%20Analysis%20Draft%3B%202020-2021.csv")

# **Data Preparation**

##Preprocessing

In [ ]:
#renaming the column names
column_mapping = {
    'player_name': 'player_name',
    'team': 'team',
    'conf': 'conference',
    'GP': 'games_played',
    'Min_per': 'min_per',
    'Ortg': 'offensive_rating',
    'usg': 'usage',
    'eFG': 'effective_field_goal_%',
    'TS_per': 'true_shooting_%',
    'ORB_per': 'offensive_rebound_%',
    'DRB_per': 'defensive_rebound_%',
    'AST_per': 'assist_%',
    'TO_per': 'turnover_%',
    'FTM': 'free_throws_made',
    'FTA': 'free_throws_attempted',
    'FT_per': 'free_throws_%',
    'twoPM': '2-pointers_made',
    'twoPA': '2-pointers_attempted',
    'twoP_per': '2-pointers_%',
    'TPM': '3-pointers_made',
    'TPA': '3-pointers_attempted',
    'TP_per': '3-pointers_%',
    'blk_per': 'Block_%',
    'stl_per': 'Steal_%',
    'ftr': 'free_throw_rate',
    'yr': 'year_of_college',
    'ht': 'ht',
    'num': 'num',
    'porpag': 'Pointers_over_replacement_per_adjusted_game',
    'adjoe': 'Adjusted_offensive_efficiency',
    'pfr': 'pfr',
    'year': 'year',
    'pid': 'Player_ID_(recurring for each year)',
    'type': 'Type',
    'Rec Rank': 'Recruit_Rank',
    'ast/tov': 'Assist / Turnover ratio',
    'rimmade': 'Rim_shots_made',
    'rimmade+rimmiss': 'Rim-shots_attempted',
    'midmade': 'Midrange_shots_made',
    'midmade+midmiss': 'Midrange_shots_attempted',
    'rimmade/(rimmade+rimmiss)': 'Rim_shots %',
    'midmade/(midmade+midmiss)': 'Midrange_shots %',
    'dunksmade': 'Dunks_made',
    'dunksmiss+dunksmade': 'Dunks_attempted',
    'dunksmade/(dunksmade+dunksmiss)': 'Dunks %',
    'pick': 'Pick_drafted',
    'drtg': 'Defensive_rating',
    'adrtg': 'Adjusted_defensive_rating',
    'dporpag': 'Defensive_points_over_replacement_per_adjusted_game',
    'stops': 'Stops',
    'bpm': 'Box plus/minus',
    'obpm': 'Offensive box plus/minus',
    'dbpm': 'Defensive box plus/minus',
    'gbpm': 'gbpm',
    'mp': 'Minutes_played',
    'ogbpm': 'Offensive_gbpm',
    'dgbpm': 'Defensive_gbpm',
    'oreb': 'Offensive_rebounds',
    'dreb': 'Defensive_rebounds',
    'treb': 'Total_rebounds',
    'ast': 'Assists',
    'stl': 'Steals',
    'blk': 'Blocks',
    'pts': 'Points',
    'Unnamed: 64': 'Role/Position',
    'Unnamed: 65': 'Unnamed: 65'
}

df.rename(columns=column_mapping, inplace=True)


In [ ]:
#overview of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61061 entries, 0 to 61060
Data columns (total 66 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   player_name                                          61061 non-null  object 
 1   team                                                 61061 non-null  object 
 2   conference                                           61061 non-null  object 
 3   games_played                                         61061 non-null  int64  
 4   min_per                                              61061 non-null  float64
 5   offensive_rating                                     61061 non-null  float64
 6   usage                                                61061 non-null  float64
 7   effective_field_goal_%                               61061 non-null  float64
 8   true_shooting_%                                      61061 non-nul

In [ ]:
#checking for unique values in pick drafted
dfdraft = df.copy()
dfdraft = dfdraft[dfdraft['Pick_drafted'].notna()]
dfdraft['Player_ID_(recurring for each year)'].nunique()

630

**1. year_of_college**

In [ ]:
df['year_of_college'].unique()

array(['So', 'Sr', 'Jr', 'Fr', '0', nan, '57.1', '42.9', 'None'],
      dtype=object)

In [ ]:
#dropping year of college ~ 0.044% of the data being dropped
drop = ['None', '0', '57.1', '42.9']
df = df[~df['year_of_college'].isin(drop)]


In [ ]:
#checking for null values
df['year_of_college'].isna().sum()

274

In [ ]:
df.dropna(subset=['year_of_college'], inplace=True)


<ipython-input-9-ed5a7f58cfee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['year_of_college'], inplace=True)


**2. HT: Dropping the column since it does not provide any relevant player performance information**


In [ ]:
#dropping ht column since its not of relevance to us
df.drop(columns=['ht'], inplace=True)


<ipython-input-10-48fe64262b7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['ht'], inplace=True)


**3. Num: Assuming it is the jersey number we decide to appoint jersey numbers for each team that have not been already taken**

In [ ]:
#checking for unique values in num column
df["num"].unique()

array([nan, '53', '5', '14', '22', '20', '34', '24', '54', '15', '13',
       '10', '21', '32', '12', '51', '31', '33', '1', '45', '50', '3',
       '23', '2', '4', '55', '35', '42', '25', '41', '52', '11', '40',
       '30', '43', '0', '44', '23B', 'None', '26', '99', '4A', '31/24',
       '--', 10.0, 22.0, 3.0, 23.0, 30.0, 13.0, 15.0, 1.0, 5.0, 35.0, 2.0,
       32.0, 11.0, 0.0, 40.0, 33.0, 4.0, 12.0, 20.0, 50.0, 25.0, 44.0,
       21.0, 31.0, 14.0, 34.0, 24.0, 55.0, 45.0, 52.0, 51.0, 41.0, 54.0,
       42.0, 43.0, 53.0], dtype=object)

In [ ]:
#replacing irregular values to nan
non_numeric_values = ['None', '--', '23B', '4A', '31/24']

df['num'] = df['num'].replace(non_numeric_values, np.nan)

# Check unique values again
print(df['num'].unique())


[nan '53' '5' '14' '22' '20' '34' '24' '54' '15' '13' '10' '21' '32' '12'
 '51' '31' '33' '1' '45' '50' '3' '23' '2' '4' '55' '35' '42' '25' '41'
 '52' '11' '40' '30' '43' '0' '44' '26' '99' 10.0 22.0 3.0 23.0 30.0 13.0
 15.0 1.0 5.0 35.0 2.0 32.0 11.0 0.0 40.0 33.0 4.0 12.0 20.0 50.0 25.0
 44.0 21.0 31.0 14.0 34.0 24.0 55.0 45.0 52.0 51.0 41.0 54.0 42.0 43.0
 53.0]


<ipython-input-12-b5e5f933b532>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num'] = df['num'].replace(non_numeric_values, np.nan)


In [ ]:
#appoint jersey numbers for each team that have not been already taken
import numpy as np

# Fill null values in 'num' column with NaN
df['num'] = df['num'].fillna(np.nan)

# Group by team and fill null values with NaN if no numbers available
for team, group in df.groupby('team'):
    team_numbers = set(group['num'].dropna().unique())
    available_numbers = [num for num in range(1, 101) if num not in team_numbers]
    df.loc[group.index, 'num'] = group['num'].apply(lambda x: x if pd.notnull(x) else next(iter(available_numbers), np.nan))








<ipython-input-13-4a670511c9c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num'] = df['num'].fillna(np.nan)


**4. Recruit Ranker**

In [ ]:
#replacing null values with 0 since not all players get ranked
df['Recruit_Rank'].isna().sum()
df['Recruit_Rank'].fillna(0, inplace=True)

<ipython-input-14-0781cde73ee0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recruit_Rank'].fillna(0, inplace=True)


**5. Assist / Turnover**

In [ ]:
#replacing null values for Assist / Turnover ratio with median
median = df['Assist / Turnover ratio'].median()
df['Assist / Turnover ratio'] = df['Assist / Turnover ratio'].fillna(median)



<ipython-input-15-0088ae74358d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Assist / Turnover ratio'] = df['Assist / Turnover ratio'].fillna(median)


**6. Searching for Nulls in Columns 46-64**

In [ ]:
#checking for null values for columns from 46 to 64
df[['Defensive_rating','Adjusted_defensive_rating', 'Defensive_points_over_replacement_per_adjusted_game', 'Stops','Box plus/minus','Offensive box plus/minus','Defensive box plus/minus','gbpm','Minutes_played','Offensive_gbpm','Defensive_gbpm','Offensive_rebounds','Defensive_rebounds','Total_rebounds','Assists','Steals','Blocks','Points']].isna().sum()

Defensive_rating                                       8
Adjusted_defensive_rating                              8
Defensive_points_over_replacement_per_adjusted_game    8
Stops                                                  8
Box plus/minus                                         8
Offensive box plus/minus                               8
Defensive box plus/minus                               8
gbpm                                                   8
Minutes_played                                         3
Offensive_gbpm                                         8
Defensive_gbpm                                         8
Offensive_rebounds                                     3
Defensive_rebounds                                     3
Total_rebounds                                         3
Assists                                                3
Steals                                                 3
Blocks                                                 3
Points                         

In [ ]:
#dropping the null values which accounts for less than 1% of the dataset
df.dropna(subset=['Defensive_rating','Adjusted_defensive_rating', 'Defensive_points_over_replacement_per_adjusted_game', 'Stops','Box plus/minus','Offensive box plus/minus','Defensive box plus/minus','gbpm','Minutes_played','Offensive_gbpm','Defensive_gbpm','Offensive_rebounds','Defensive_rebounds','Total_rebounds','Assists','Steals','Blocks','Points'], inplace=True)


<ipython-input-17-6d53ea9d9dee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Defensive_rating','Adjusted_defensive_rating', 'Defensive_points_over_replacement_per_adjusted_game', 'Stops','Box plus/minus','Offensive box plus/minus','Defensive box plus/minus','gbpm','Minutes_played','Offensive_gbpm','Defensive_gbpm','Offensive_rebounds','Defensive_rebounds','Total_rebounds','Assists','Steals','Blocks','Points'], inplace=True)


**6. Searching for Nulls in Columns 36-44**

In [ ]:
#checking for null values for columns 36 to 44
#these nulls are a significantly larger number than what we are used to
df[['Rim_shots_made','Rim-shots_attempted', 'Midrange_shots_made', 'Midrange_shots_attempted','Rim_shots %','Midrange_shots %','Dunks_made','Dunks_attempted','Dunks %']].isna().sum()

Rim_shots_made               6207
Rim-shots_attempted          6207
Midrange_shots_made          6207
Midrange_shots_attempted     6207
Rim_shots %                  9923
Midrange_shots %            10180
Dunks_made                   6207
Dunks_attempted              6207
Dunks %                     33214
dtype: int64

**7. Searching for unique values in the Object Columns**

In [ ]:
df['Role/Position'].unique()

array([nan, 'Combo G', 'Pure PG', 'Wing F', 'PF/C', 'Wing G', 'C',
       'Stretch 4', 'Scoring PG'], dtype=object)

In [ ]:
df['conference'].unique()

array(['SB', 'WAC', 'BE', 'WCC', 'BW', 'Horz', 'ind', 'A10', 'NEC', 'ACC',
       'SEC', 'AE', 'MAAC', 'Pat', 'SWAC', 'CAA', 'MVC', 'MAC', 'B12',
       'CUSA', 'P10', 'B10', 'SC', 'MEAC', 'Slnd', 'Ivy', 'Sum', 'ASun',
       'MWC', 'BSky', 'OVC', 'BSth', 'GWC', 'P12', 'Amer', 'Ind'],
      dtype=object)

In [ ]:
df['Type'].unique()

array(['all'], dtype=object)

We drop the column 'Unnamed: 65' as it can not be interpreted

In [ ]:
#dropping Unnamed: 65
df.drop(columns=['Unnamed: 65'], inplace=True)

<ipython-input-22-f266d864f9c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Unnamed: 65'], inplace=True)


**8. Observing patterns in the Nulls of the ``Role/Position`` column**

We temporarily create a DataFrame called 'dfrp' to observe any patterns in the rows of players with null values in the 'Role/Position' column.

In [ ]:
dfrp = df.copy()

In [ ]:
dfrp['Role/Position'].unique()

array([nan, 'Combo G', 'Pure PG', 'Wing F', 'PF/C', 'Wing G', 'C',
       'Stretch 4', 'Scoring PG'], dtype=object)

In [ ]:
dfrp = dfrp[dfrp['Role/Position'].isna()]

In [ ]:
dfrp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4596 entries, 0 to 59476
Data columns (total 64 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   player_name                                          4596 non-null   object 
 1   team                                                 4596 non-null   object 
 2   conference                                           4596 non-null   object 
 3   games_played                                         4596 non-null   int64  
 4   min_per                                              4596 non-null   float64
 5   offensive_rating                                     4596 non-null   float64
 6   usage                                                4596 non-null   float64
 7   effective_field_goal_%                               4596 non-null   float64
 8   true_shooting_%                                      4596 non-null 

We notice that most nulls are in the column indexes from 35-43.

We drop the nulls from the 'Role/Position' by omitting their values in the below code block

In [ ]:
df = df[df['Role/Position'].isin( ['Combo G', 'Pure PG', 'Wing F', 'PF/C', 'Wing G', 'C', 'Stretch 4', 'Scoring PG'])]

**9. Observing patterns in the Nulls of the columns**

``Rim_shots_made``

``Rim-shots_attempted``

``Midrange_shots_made``

``Midrange_shots_attempted``

``Rim_shots %``

``Midrange_shots %``

``Dunks_made``

``Dunks_attempted``

``Dunks %'``


We decide to observe patterns in the missing values. We create a temporary dataframe called 'dfpractice'.

In [ ]:
dfpractice = df[['Rim_shots_made',
'Rim-shots_attempted',
'Midrange_shots_made',
'Midrange_shots_attempted',
'Rim_shots %',
'Midrange_shots %',
'Dunks_made',
'Dunks_attempted',
'Dunks %']             ]

In [ ]:
dfpracticetwo = dfpractice.copy()

'Dunks_made' is our test column. We subset the null values to see if certain values are consistently missing.

In [ ]:
dfpractice = dfpracticetwo[dfpracticetwo['Dunks_made'].isna()]

In [ ]:
dfpractice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1616 entries, 4592 to 61060
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rim_shots_made            0 non-null      float64
 1   Rim-shots_attempted       0 non-null      float64
 2   Midrange_shots_made       0 non-null      float64
 3   Midrange_shots_attempted  0 non-null      float64
 4   Rim_shots %               0 non-null      float64
 5   Midrange_shots %          0 non-null      float64
 6   Dunks_made                0 non-null      float64
 7   Dunks_attempted           0 non-null      float64
 8   Dunks %                   0 non-null      float64
dtypes: float64(9)
memory usage: 126.2 KB


We use the 'Rim_shots_made','Rim-shots_attempted', 'Rim_shots %' columns to observe the relationship between the absolute values and the percentage columns of the same performance metric. This relationship is found for:-

1. 'Rim_shots_made', 'Rim-shots_attempted', 'Rim_shots %'

2. 'Midrange_shots_made', 'Midrange_shots_attempted', 'Midrange_shots %'

3. 'Dunks_made', 'Dunks_attempted', 'Dunks %'

In [ ]:
dfrim = df[['Rim_shots_made','Rim-shots_attempted', 'Rim_shots %']]

In [ ]:
dfrim['rimdivided'] = dfrim['Rim_shots_made'] / dfrim['Rim-shots_attempted']

<ipython-input-33-593775140539>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrim['rimdivided'] = dfrim['Rim_shots_made'] / dfrim['Rim-shots_attempted']


In [ ]:
dfrim

,Rim_shots_made,Rim-shots_attempted,Rim_shots %,rimdivided
4589,48.0,75.0,0.6400,0.64000
4590,0.0,0.0,NaN,NaN
4591,82.0,134.0,0.6119,0.61194
4592,NaN,NaN,NaN,NaN
4593,NaN,NaN,NaN,NaN
...,...,...,...,...
61056,NaN,NaN,NaN,NaN
61057,0.0,0.0,NaN,NaN
61058,NaN,NaN,NaN,NaN
61059,NaN,NaN,NaN,NaN


In [ ]:
df = df[~df[['Rim_shots_made','Rim-shots_attempted','Midrange_shots_made','Midrange_shots_attempted','Dunks_made','Dunks_attempted']].isna().any(axis=1)]

**10. Treatment of columns**

We fill the columns below with 0.

In [ ]:
df[['Rim_shots %', 'Midrange_shots %', 'Dunks %']] = df[['Rim_shots %', 'Midrange_shots %', 'Dunks %']].fillna(0)

<ipython-input-36-08c811721aaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Rim_shots %', 'Midrange_shots %', 'Dunks %']] = df[['Rim_shots %', 'Midrange_shots %', 'Dunks %']].fillna(0)


we drop the 'Type' and 'num' values after seeing that they do not reflect either performance or tell us anything about the player that is not arbitrary



In [ ]:
df.drop(['Type', 'num'], axis=1, inplace=True)


<ipython-input-37-f106f7f539ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Type', 'num'], axis=1, inplace=True)


**11. Finding the unique player values**

In [ ]:
df['player_name'].duplicated().sum()

31292

The below code tells us that there are only 23230 unique players in the dataset.

In [ ]:
df['player_name'].nunique()

23247

In [ ]:
df['Player_ID_(recurring for each year)'].nunique()

23230

**12. Subsetting Performance and Player Detail Columns**

We subset the performance metrics of players below.

In [ ]:
performancecolumns = [
    'Assist / Turnover ratio',
    'Rim_shots_made',
    'Rim-shots_attempted',
    'Midrange_shots_made',
    'Midrange_shots_attempted',
    'Rim_shots %',
    'Midrange_shots %',
    'Dunks_made',
    'Dunks_attempted',
    'Dunks %',
    'Defensive_rating',
    'Adjusted_defensive_rating',
    'Defensive_points_over_replacement_per_adjusted_game',
    'Stops',
    'Box plus/minus',
    'Offensive box plus/minus',
    'Defensive box plus/minus',
    'gbpm',
    'Minutes_played',
    'Offensive_gbpm',
    'Defensive_gbpm',
    'Offensive_rebounds',
    'Defensive_rebounds',
    'Total_rebounds',
    'Assists',
    'Steals',
    'Blocks',
    'Points',
    'Pointers_over_replacement_per_adjusted_game',
    'Adjusted_offensive_efficiency',
    'pfr',
    'games_played',
    'min_per',
    'offensive_rating',
    'usage',
    'effective_field_goal_%',
    'true_shooting_%',
    'offensive_rebound_%',
    'defensive_rebound_%',
    'assist_%',
    'turnover_%',
    'free_throws_made',
    'free_throws_attempted',
    'free_throws_%',
    '2-pointers_made',
    '2-pointers_attempted',
    '2-pointers_%',
    '3-pointers_made',
    '3-pointers_attempted',
    '3-pointers_%',
    'Block_%',
    'Steal_%',
    'free_throw_rate'
]

We make a list of the columns that give us player details. We use the first occurring instance of each column.

In [ ]:
newdfplayerdetails = ['player_name','team','conference','Role/Position','year_of_college','year','Recruit_Rank','Role/Position','Pick_drafted']

**13. Creating the cleaned DataFrame**

In [ ]:
agg_dict = {**dict.fromkeys(performancecolumns, 'mean'), **dict.fromkeys(newdfplayerdetails, 'first')}

In [ ]:
newdf = df.groupby('Player_ID_(recurring for each year)').agg(agg_dict).reset_index()

In [ ]:
newdf = newdf.iloc[:,[54,55,56,57,58,59,60,61,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,\
              35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53]]

In [ ]:
newdf = newdf.drop(['player_name', 'team','conference','Role/Position','year_of_college','year','Player_ID_(recurring for each year)'], axis=1)

In [ ]:
newdf['Pick_drafted'].isnull().sum()

22648